In [1]:
from pymongo import MongoClient
import sys
sys.path.append('..')
from calcimetry.mongo_api import MongoInfo, MongoAPI
import calcimetry.use_server as server
img_path = server.init()
import pandas as pd
import json
import numpy as np
import re
import os

server : http://imgserver.10.68.0.250.nip.io/data/andra


In [5]:
dirhead = '/work/armitagj/data/csvs/REP4/'

drillfiles = [ f.path for f in os.scandir(dirhead) if f.is_dir() ]
filename = 'imgs2.csv'
mesuname = 'mesu.csv'

# loop through all directories on the local system
for i, drill in enumerate(drillfiles):

    if i == 0:
        # grab the only diretory (not always called "Photos")
        images = [ f.path for f in os.scandir(drill) if f.is_dir() ]

        print(images)
        csvfile = images[0] + '/' + filename
        calcifile = images[0] + '/' + mesuname

        df = pd.read_csv(calcifile, delimiter=';')

['/work/armitagj/data/csvs/REP4/BPE4023/Photos']


In [6]:
df

,MeasureId,ImageId,CalciCote,CalciVals1m,CalciVals15m
0,0,0,65,31,33
1,1,0,75,33,35
2,2,0,85,33,36
3,3,0,95,33,36
4,4,0,105,32,33
5,5,1,95,33,36
6,6,1,105,32,33
7,7,1,115,32,33
8,8,1,125,32,33
9,9,1,134,32,33


In [9]:
print(drill.split('/')[-1])

TPV1322


In [10]:
import glob

In [11]:
glob.glob('/work/armitagj/data/Excell_and_Photos/BPE4023/*.csv')

['/work/armitagj/data/Excell_and_Photos/BPE4023/GTR2012-07_0023_BPE4023_Calcimétrie.csv']

In [49]:
drill = '/work/armitagj/data/csvs/REP4/TPV1322'
images = '/work/armitagj/data/csvs/REP4/TPV1322/Photos'
calcihead = '/work/armitagj/data/Excell_and_Photos/'

csvfile = images + '/' + filename
calcifile = images + '/' + mesuname

excell2csved = glob.glob(calcihead + drill.split('/')[-1] + '/*.csv')

print(excell2csved)

df1 = pd.read_csv(excell2csved[0],sep=',')
df2 = pd.read_csv(csvfile,sep=',')

id = np.nan

with open(calcifile, 'w') as g:
    g.write("MeasureId;ImageId;CalciCote;CalciVals1m;CalciVals15m\n")
    for i, j in df2.iterrows():
        imgId,cote0,cote1 = j["ImageId"],int(round(float(j["Cote0"]))),int(round(float(j["Cote1"])))
        for k1, k2 in df1.iterrows():
            # c = int(round(float(k2["Cote"].replace(',','.'))*100))
            c = k2["Cote"] * 100
            if cote0<=c and c<=cote1:
                g.write(str(id)+";"+str(imgId)+";"+str(c)+";"+str(df1.iloc[k1, 1])+";"+str(df1.iloc[k1, 3])+"\n")
    g.close()

# read file
df = pd.read_csv(calcifile, delimiter=';')

# create the json payload
payload = json.loads(df.to_json(orient='records'))

['/work/armitagj/data/Excell_and_Photos/TPV1322/GTR2012-12_0034_TPV1322_Calcimétrie.csv']


In [50]:
mongo_info = MongoInfo()

drillname = drill.split('/')[-1]

with MongoAPI(mongo_info=mongo_info) as mongo_api:
    partialdrillname = f'.*{drillname}.*'
    print(partialdrillname)
    doc = mongo_api.db['images'].find_one({'DrillName': {'$regex' : partialdrillname}})

.*TPV1322.*


In [51]:
print(doc['DrillName'])

TPV1322


In [52]:
df['DrillName'] = drill.split('/')[-1]
df

,MeasureId,ImageId,CalciCote,CalciVals1m,CalciVals15m,DrillName
0,NaN,2868,38.0,0.38,20,TPV1322
1,NaN,2868,51.0,0.51,20,TPV1322
2,NaN,2868,61.0,0.61,18,TPV1322
3,NaN,2868,72.0,0.72,22,TPV1322
4,NaN,2868,84.0,0.84,22,TPV1322
5,NaN,2868,94.0,0.94,22,TPV1322
6,NaN,2868,106.0,1.06,22,TPV1322
7,NaN,2869,84.0,0.84,22,TPV1322
8,NaN,2869,94.0,0.94,22,TPV1322
9,NaN,2869,106.0,1.06,22,TPV1322


In [39]:
payload = json.loads(df.to_json(orient='records'))

In [ ]:
with MongoAPI(mongo_info=mongo_info) as mongo_api:
                        if mongo_api.db['mesurements'].find_one(
                                {'DrillName': df['DrillName'][0]}) is None:
                            mongo_api.write_img_many(payload)